## Rag with Gemini

In [1]:
import textwrap
import chromadb
import numpy as np
import pandas as pd

from IPython.display import Markdown
from chromadb import Documents, EmbeddingFunction, Embeddings
from chromadb.config import Settings

In [2]:
from google import genai
from dotenv import load_dotenv
import os
load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")
client = genai.Client(api_key=api_key)
print("Google GenAI client initialized.")

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


Google GenAI client initialized.


In [3]:
for m in client.models.list():
  if 'embedContent' in m.supported_actions:
    print(m.name)

models/embedding-001
models/text-embedding-004
models/gemini-embedding-exp-03-07
models/gemini-embedding-exp


In [4]:
## Test Document Embedding

DOCUMENT1 = """
  Operating the Climate Control System  Your Googlecar has a climate control
  system that allows you to adjust the temperature and airflow in the car.
  To operate the climate control system, use the buttons and knobs located on
  the center console.  Temperature: The temperature knob controls the
  temperature inside the car. Turn the knob clockwise to increase the
  temperature or counterclockwise to decrease the temperature.
  Airflow: The airflow knob controls the amount of airflow inside the car.
  Turn the knob clockwise to increase the airflow or counterclockwise to
  decrease the airflow. Fan speed: The fan speed knob controls the speed
  of the fan. Turn the knob clockwise to increase the fan speed or
  counterclockwise to decrease the fan speed.
  Mode: The mode button allows you to select the desired mode. The available
  modes are: Auto: The car will automatically adjust the temperature and
  airflow to maintain a comfortable level.
  Cool: The car will blow cool air into the car.
  Heat: The car will blow warm air into the car.
  Defrost: The car will blow warm air onto the windshield to defrost it.
"""

In [5]:
from google.genai import types

class GeminiEmbeddingFunction(EmbeddingFunction):
  def __call__(self, input: Documents) -> Embeddings:
    EMBEDDING_MODEL_ID = "models/text-embedding-004"  # @param ["models/embedding-001", "models/text-embedding-004", "models/gemini-embedding-exp-03-07", "models/gemini-embedding-exp"] {"allow-input": true, "isTemplate": true}
    title = "Custom query"
    response = client.models.embed_content(
        model=EMBEDDING_MODEL_ID,
        contents=input,
        config=types.EmbedContentConfig(
          task_type="retrieval_document",
          title=title
        )
    )

    return response.embeddings[0].values

In [6]:
def create_chroma_db(documents, name, persist_dir="chroma_data"):
    # Use PersistentClient to save the DB to disk
    chroma_client = chromadb.PersistentClient(
        path=persist_dir,
        settings=Settings()
    )

    # Create or load the collection
    db = chroma_client.get_or_create_collection(
        name=name,
        embedding_function=GeminiEmbeddingFunction()
    )

    # Add documents to the collection
    for i, d in enumerate(documents):
        db.add(
            documents=[d],  # List of one document
            ids=[str(i)]     # List of one ID
        )

    # No need to call chroma_client.persist() – saving is automatic
    return chroma_client, db

In [7]:
# Create and persist your ChromaDB locally
documents = [DOCUMENT1]
chroma_client, db = create_chroma_db(documents, "googlecarsdatabase", persist_dir="chroma_data")

C:\Users\12345\AppData\Local\Temp\ipykernel_756\851126105.py:11: DeprecationWarning: The class GeminiEmbeddingFunction does not implement __init__. This will be required in a future version.
  embedding_function=GeminiEmbeddingFunction()


In [8]:
sample_data = db.get(include=['documents', 'embeddings'])

df = pd.DataFrame({
    "IDs": sample_data['ids'][:3],
    "Documents": sample_data['documents'][:3],
    "Embeddings": [str(emb)[:50] + "..." for emb in sample_data['embeddings'][:3]]  # Truncate embeddings
})

print(df)

  IDs                                          Documents  \
0   0  \n  Operating the Climate Control System  Your...   

                                          Embeddings  
0  [ 2.09244359e-02  1.38586573e-02 -3.33479755e-...  


In [9]:
def get_relevant_passage(query, db):
  passage = db.query(query_texts=[query], n_results=1)['documents'][0][0]
  return passage

In [10]:
# Perform embedding search
passage = get_relevant_passage("touch screen features", db)
Markdown(passage)


  Operating the Climate Control System  Your Googlecar has a climate control
  system that allows you to adjust the temperature and airflow in the car.
  To operate the climate control system, use the buttons and knobs located on
  the center console.  Temperature: The temperature knob controls the
  temperature inside the car. Turn the knob clockwise to increase the
  temperature or counterclockwise to decrease the temperature.
  Airflow: The airflow knob controls the amount of airflow inside the car.
  Turn the knob clockwise to increase the airflow or counterclockwise to
  decrease the airflow. Fan speed: The fan speed knob controls the speed
  of the fan. Turn the knob clockwise to increase the fan speed or
  counterclockwise to decrease the fan speed.
  Mode: The mode button allows you to select the desired mode. The available
  modes are: Auto: The car will automatically adjust the temperature and
  airflow to maintain a comfortable level.
  Cool: The car will blow cool air into the car.
  Heat: The car will blow warm air into the car.
  Defrost: The car will blow warm air onto the windshield to defrost it.


In [11]:
def make_prompt(query, relevant_passage):
  escaped = relevant_passage.replace("'", "").replace('"', "").replace("\n", " ")
  prompt = ("""
    You are a helpful and informative bot that answers questions using
    text from the reference passage included below.
    Be sure to respond in a complete sentence, being comprehensive,
    including all relevant background information.
    However, you are talking to a non-technical audience, so be sure to
    break down complicated concepts and strike a friendly
    and converstional tone. If the passage is irrelevant to the answer,
    you may ignore it.
    QUESTION: '{query}'
    PASSAGE: '{relevant_passage}'

    ANSWER:
  """).format(query=query, relevant_passage=escaped)

  return prompt

In [12]:
query = "How do you use the touchscreen in the Google car?"
prompt = make_prompt(query, passage)
Markdown(prompt)


    You are a helpful and informative bot that answers questions using
    text from the reference passage included below.
    Be sure to respond in a complete sentence, being comprehensive,
    including all relevant background information.
    However, you are talking to a non-technical audience, so be sure to
    break down complicated concepts and strike a friendly
    and converstional tone. If the passage is irrelevant to the answer,
    you may ignore it.
    QUESTION: 'How do you use the touchscreen in the Google car?'
    PASSAGE: '   Operating the Climate Control System  Your Googlecar has a climate control   system that allows you to adjust the temperature and airflow in the car.   To operate the climate control system, use the buttons and knobs located on   the center console.  Temperature: The temperature knob controls the   temperature inside the car. Turn the knob clockwise to increase the   temperature or counterclockwise to decrease the temperature.   Airflow: The airflow knob controls the amount of airflow inside the car.   Turn the knob clockwise to increase the airflow or counterclockwise to   decrease the airflow. Fan speed: The fan speed knob controls the speed   of the fan. Turn the knob clockwise to increase the fan speed or   counterclockwise to decrease the fan speed.   Mode: The mode button allows you to select the desired mode. The available   modes are: Auto: The car will automatically adjust the temperature and   airflow to maintain a comfortable level.   Cool: The car will blow cool air into the car.   Heat: The car will blow warm air into the car.   Defrost: The car will blow warm air onto the windshield to defrost it. '

    ANSWER:
  

In [13]:
MODEL_ID = "gemini-2.0-flash"  # @param ["gemini-2.0-flash-lite", "gemini-2.0-flash", "gemini-2.5-flash-preview-05-20","gemini-2.5-pro-preview-05-06"] {"allow-input": true, "isTemplate": true}
answer = client.models.generate_content(
    model = MODEL_ID,
    contents = prompt
)
Markdown(answer.text)

I am sorry, but the provided passage does not mention how to use the touchscreen in the Google car, it only provides instructions on how to operate the climate control system using the buttons and knobs located on the center console.


In [14]:
# Load existing persistent collection
chroma_client = chromadb.PersistentClient(path="chroma_data", settings=Settings())
db = chroma_client.get_or_create_collection(
    name="googlecarsdatabase",
    embedding_function=GeminiEmbeddingFunction()
)

C:\Users\12345\AppData\Local\Temp\ipykernel_756\1414569679.py:5: DeprecationWarning: The class GeminiEmbeddingFunction does not implement __init__. This will be required in a future version.
  embedding_function=GeminiEmbeddingFunction()


## Rag with Langchain + Gemini

In [15]:
MODEL_ID = "gemini-2.0-flash"  # @param ["gemini-2.0-flash-lite", "gemini-2.0-flash", "gemini-2.5-flash-preview-05-20","gemini-2.5-pro-preview-05-06"] {"allow-input": true, "isTemplate": true}

In [18]:
from langchain_google_genai import ChatGoogleGenerativeAI
chat_model = ChatGoogleGenerativeAI(google_api_key=api_key, 
                                   model=MODEL_ID,)

In [20]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("../data/raw/General Notes.pdf")
pages = loader.load_and_split()

In [25]:
len(pages), print(pages[0].page_content[:500])  # Check the first page content

GENERAL RULES OF INTERPRETATION
Classification of goods in the tariff schedule shall be governed by the following principles:
1. The table of contents, alphabetical index, and titles of sections, chapters and sub-chapters are provided for ease of reference only;
for legal purposes, classification shall be determined according to the terms of the headings and any relative section or chapter
notes and, provided such headings or notes do not otherwise require, according to the following provisions:


(1045, None)

In [26]:
# Split the document into chunks

from langchain_text_splitters import NLTKTextSplitter

text_splitter = NLTKTextSplitter(chunk_size=500, chunk_overlap=100)

chunks = text_splitter.split_documents(pages)

print(len(chunks))

print(type(chunks[0]))

Created a chunk of size 1091, which is longer than the specified 500
Created a chunk of size 706, which is longer than the specified 500
Created a chunk of size 1409, which is longer than the specified 500
Created a chunk of size 1117, which is longer than the specified 500
Created a chunk of size 1234, which is longer than the specified 500
Created a chunk of size 673, which is longer than the specified 500
Created a chunk of size 873, which is longer than the specified 500
Created a chunk of size 573, which is longer than the specified 500
Created a chunk of size 1052, which is longer than the specified 500
Created a chunk of size 1216, which is longer than the specified 500
Created a chunk of size 592, which is longer than the specified 500
Created a chunk of size 1175, which is longer than the specified 500
Created a chunk of size 501, which is longer than the specified 500
Created a chunk of size 810, which is longer than the specified 500
Created a chunk of size 529, which is lon

6655
<class 'langchain_core.documents.base.Document'>


In [28]:
# Creating Chunks Embedding

from langchain_google_genai import GoogleGenerativeAIEmbeddings

embedding_model = GoogleGenerativeAIEmbeddings(google_api_key=api_key, model="models/text-embedding-004")

In [29]:
# Store the chunks in vector store
from langchain_community.vectorstores import Chroma

# Embed each chunk and load it into the vector store
db = Chroma.from_documents(chunks, embedding_model, persist_directory="./chroma_db_")

# Persist the database on drive
db.persist()

C:\Users\12345\AppData\Local\Temp\ipykernel_756\484647787.py:8: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


In [30]:
# Setting a Connection with the ChromaDB
db_connection = Chroma(persist_directory="./chroma_db_", embedding_function=embedding_model)

C:\Users\12345\AppData\Local\Temp\ipykernel_756\192857647.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db_connection = Chroma(persist_directory="./chroma_db_", embedding_function=embedding_model)


In [31]:
# Converting CHROMA db_connection to Retriever Object
retriever = db_connection.as_retriever(search_kwargs={"k": 5})

print(type(retriever))

<class 'langchain_core.vectorstores.base.VectorStoreRetriever'>


In [32]:
from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate

In [36]:
chat_template = ChatPromptTemplate.from_messages([
    # System Message Prompt Template
    SystemMessage(content="""You are TariffBot — an intelligent assistant trained on U.S. International Trade Commission data.
 You exist to help importers, analysts, and trade professionals quickly understand tariff rules, duty rates, and policy agreements.
You always provide clear, compliant, and factual answers grounded in official HTS documentation.


When given an HTS code and product information, you explain all applicable duties and cost components.


When asked about trade agreements (e.g., NAFTA, Israel FTA), you reference the relevant General Notes with citations.


If a query is ambiguous or unsupported, you politely defer or recommend reviewing the relevant HTS section manually.


You do not speculate or make policy interpretations — you clarify with precision and data.
"""),
    # Human Message Prompt Template
    HumanMessagePromptTemplate.from_template("""Answer the question based on the given context.
    Context: {context}
    Question: {question}
    Answer: """)
])

In [37]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [38]:
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | chat_template
    | chat_model
    | output_parser
)

In [ ]:
response = rag_chain.invoke("""What is the United States-Israel Free Trade Agreement?
""")

'The United States-Israel Free Trade Area Implementation Act of 1985 states that products of Israel described in Annex 1 of the agreement are subject to duty as provided in the act. Certain articles imported directly from the West Bank, the Gaza Strip, a qualifying industrial zone, or Israel may be eligible for duty-free entry if they meet specific criteria, including being wholly grown, produced, or manufactured in those areas or being new or different articles of commerce with a minimum of 35 percent of their appraised value originating from those areas or Israel. Goods are eligible for treatment as "products of Israel" if they meet specific requirements, including being the growth, product, or manufacture of Israel, being imported directly from Israel (or the West Bank, Gaza Strip, or a qualifying industrial zone), and having a minimum of 35 percent of their appraised value originating from materials and processing in Israel (including the West Bank, Gaza Strip, or a qualifying indu

In [40]:
Markdown(response)

The United States-Israel Free Trade Area Implementation Act of 1985 states that products of Israel described in Annex 1 of the agreement are subject to duty as provided in the act. Certain articles imported directly from the West Bank, the Gaza Strip, a qualifying industrial zone, or Israel may be eligible for duty-free entry if they meet specific criteria, including being wholly grown, produced, or manufactured in those areas or being new or different articles of commerce with a minimum of 35 percent of their appraised value originating from those areas or Israel. Goods are eligible for treatment as "products of Israel" if they meet specific requirements, including being the growth, product, or manufacture of Israel, being imported directly from Israel (or the West Bank, Gaza Strip, or a qualifying industrial zone), and having a minimum of 35 percent of their appraised value originating from materials and processing in Israel (including the West Bank, Gaza Strip, or a qualifying industrial zone).

In [41]:
response = rag_chain.invoke("""Can a product that exceeds its tariff-rate quota still qualify for duty-free entry under GSP or any FTA? Why or why not?
""")
Markdown(response)    

A product that exceeds its tariff-rate quota cannot qualify for duty-free entry under the CBERA, according to General Note 7(g). Additionally, exceeding the in-quota quantity disqualifies agricultural products in chapters 2 through 52 from duty-free treatment under CBERA.